In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
import animalshelter.animal_shelter_crud as crud

# ensuring I receive the latest updates to from my python module
import importlib
importlib.reload(crud)

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Animals4Austin"

# Connect to database via CRUD Module
db = crud.AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GSLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.P('Select your preferred filter option. No option returns list of all animals.'),
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.Dropdown(['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster Rescue or Individual Tracking'],
                'Select...', id='filter-type'),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                        # cutting off data cells to keep width of table within its container
                        style_cell={
                            'overflow':'hidden',
                            'textOverflow':'ellipsis',
                            'maxWidth':0
                        },

                        # tooltip to allow users to hover over column headers to receive full information
                        tooltip_header={i: i for i in df.columns},

                        # tooltip to allow users to hover over cells to receive full information
                        tooltip_data=[
                            {
                                column: {'value':str(value), 'type':'markdown'}
                                for column, value in row.items()
                            } for row in df.to_dict('records')
                        ],
                        # no duration on tooltip hover
                        tooltip_duration=None,
                        # table is not editable
                        editable=False,
                        # dash takes care of filtering via data provided in the table
                        filter_action='native',
                        # dash takes care of sorting via data provided in the table
                        sort_action='native',
                        # users can sort table through multiple columns
                        sort_mode='multi',
                        # row selectable set to single for mapping callback to function properly
                        row_selectable='single',
                        # because columns is initialized with selectable as true, column_selectable prop is set to single to
                        # allow users to change column selection for only one column at a time
                        column_selectable='single',
                        # selected rows defined with first row index to ensure geolocation chart shows one row if user does not
                        # click a row
                        selected_rows=[0],
                        # selected_columns defined with a starting column to prevent callback error with update_styles func
                        selected_columns=['rec_num'],
                        # dash takes care of paging via data in table where not all rows will be displayed at once
                        page_action='native',
                        # page size of 10 rows
                        page_size=10
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Br(),
    html.Hr(),
    # nesting the Img html componenet within an Anchor component to redirect users to website
    html.A(
        href='https://www.snhu.edu',
        target='_blank',
        children=[
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    alt='Grazioso Salvare Logo')
        ]),
    # Unique identifier with my name
    html.H1('Grazioso Salvare Dashboard by Finnegan Thomas')
])

#############################################
# Interaction Between Components / Controller
#############################################




# adding additional output because codebase returns two variables
@app.callback(Output('datatable-id','data'),
              Output('datatable-id', 'columns'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
# using if statements to decide which query to activate via filter type
    if filter_type == 'Water Rescue':
        # query for water rescue dogs
        query = {'animal_type':'Dog',
                'breed':{'$in':['Labrador Retriever Mix',
                                'Chesapeake Bay Retriever',
                                'Newfoundland']},
                'sex_upon_outcome':'Intact Female',
                'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}
                }
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {'animal_type':'Dog',
                 'breed':{'$in':['German Shepherd',
                                'Alaskan Malamute',
                                'Old English Sheepdog',
                                'Siberian Husky',
                                'Rottweiler']},
                 'sex_upon_outcome':'Intact Male',
                 'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}
                }
    elif filter_type == 'Disaster Rescue or Individual Tracking':
        query = {'animal_type':'Dog',
                 'breed':{'$in':['Doberman Pinscher',
                                'German Sheperd',
                                'Golden Retriever',
                                'Bloodhound',
                                'Rottweiler']},
                 'sex_upon_outcome':'Intact Male',
                 'age_upon_outcome_in_weeks':{'$gte':20,'$lte':300}
                }
    else:
        query = {}
    
    df = pd.DataFrame.from_records(db.read(query))
    # remember to drop id column to prevent errors
    df.drop(columns=['_id'],inplace=True)
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
       
       
    return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.histogram(dff,
                                  x='breed',
                                  title='Breed Distribution')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:14549/
